In [15]:
import pandas as pd
import numpy as np
dataset = pd.read_csv('DataSets/train.csv', delimiter=';')
dataset_gpt1 = pd.read_csv('DataSets/gpt1.csv', delimiter=';')
dataset_gpt2 = pd.read_csv('DataSets/gpt2.csv', delimiter=';')
dataset_gpt3 = pd.read_csv('DataSets/gpt3.csv', delimiter=';')
dataset_gpt4 = pd.read_csv('DataSets/gpt4.csv', delimiter=';')
dataset_gpt5 = pd.read_csv('DataSets/gpt5.csv', delimiter=';')
dataset_gpt6 = pd.read_csv('DataSets/gpt6.csv', delimiter=';')
test_count = 6000

In [16]:
import pymorphy3
from razdel import tokenize
import nltk
from nltk.corpus import stopwords

nltk.download("stopwords")
russian_stopwords = stopwords.words("russian")

morph = pymorphy3.MorphAnalyzer()

stopTags = ['PNCT', 'NUMB', 'UNKN', 'LATN', 'ROMN']

def preprocess_text(text):
    tokens = tokenize(text.lower())
    tokens = [morph.parse(token.text)[0].normal_form for token in tokens 
              if morph.parse(token.text)[0].normal_form not in russian_stopwords and 
              not any(tag in morph.parse(token.text)[0].tag for tag in stopTags)]
    text = " ".join(tokens) 
    return text


dataset = dataset.drop_duplicates()
dataset = dataset.dropna()
dataset = dataset[dataset['Текст инцидента'] != '\'Сообщение без текста']
for msg in dataset['Текст инцидента']:
    if len(msg.split(' ')) <= 2:    
        dataset = dataset[dataset['Текст инцидента'] != msg]

dataset_test = dataset.iloc[:test_count, :]
dataset_train = dataset.iloc[test_count:, :]


dataset_gpt1 = dataset_gpt1.drop_duplicates()
dataset_gpt1 = dataset_gpt1.dropna()
dataset_gpt1 = dataset_gpt1[dataset_gpt1['Текст инцидента'] != '\'Сообщение без текста']
for msg in dataset_gpt1['Текст инцидента']:
    if len(msg.split(' ')) <= 2:    
        dataset_gpt1 = dataset_gpt1[dataset_gpt1['Текст инцидента'] != msg]
        
dataset_gpt2 = dataset_gpt2.drop_duplicates()
dataset_gpt2 = dataset_gpt2.dropna()
dataset_gpt2 = dataset_gpt2[dataset_gpt2['Текст инцидента'] != '\'Сообщение без текста']
for msg in dataset_gpt2['Текст инцидента']:
    if len(msg.split(' ')) <= 2:    
        dataset_gpt2 = dataset_gpt2[dataset_gpt2['Текст инцидента'] != msg]

dataset_gpt3 = dataset_gpt3.drop_duplicates()
dataset_gpt3 = dataset_gpt3.dropna()
dataset_gpt3 = dataset_gpt3[dataset_gpt3['Текст инцидента'] != '\'Сообщение без текста']
for msg in dataset_gpt3['Текст инцидента']:
    if len(msg.split(' ')) <= 2:    
        dataset_gpt3 = dataset_gpt3[dataset_gpt3['Текст инцидента'] != msg]

dataset_gpt4 = dataset_gpt4.drop_duplicates()
dataset_gpt4 = dataset_gpt4.dropna()
dataset_gpt4 = dataset_gpt4[dataset_gpt4['Текст инцидента'] != '\'Сообщение без текста']
for msg in dataset_gpt4['Текст инцидента']:
    if len(msg.split(' ')) <= 2:    
        dataset_gpt4 = dataset_gpt4[dataset_gpt4['Текст инцидента'] != msg]

dataset_gpt5 = dataset_gpt5.drop_duplicates()
dataset_gpt5 = dataset_gpt5.dropna()
dataset_gpt5 = dataset_gpt5[dataset_gpt5['Текст инцидента'] != '\'Сообщение без текста']
for msg in dataset_gpt5['Текст инцидента']:
    if len(msg.split(' ')) <= 2:    
        dataset_gpt5 = dataset_gpt5[dataset_gpt5['Текст инцидента'] != msg]

dataset_gpt6 = dataset_gpt6.drop_duplicates()
dataset_gpt6 = dataset_gpt6.dropna()
dataset_gpt6 = dataset_gpt6[dataset_gpt6['Текст инцидента'] != '\'Сообщение без текста']
for msg in dataset_gpt6['Текст инцидента']:
    if len(msg.split(' ')) <= 2:    
        dataset_gpt6 = dataset_gpt6[dataset_gpt6['Текст инцидента'] != msg]
        

dataset_train = dataset_train._append(dataset_gpt1)
dataset_train = dataset_train._append(dataset_gpt2)
dataset_train = dataset_train._append(dataset_gpt3)
dataset_train = dataset_train._append(dataset_gpt4)
dataset_train = dataset_train._append(dataset_gpt5)
dataset_train = dataset_train._append(dataset_gpt6)

def get_group_code():
    groups = dataset['Группа тем'].unique()
    dict = {groups[i]:i for i in range(0, len(groups))}
    return dict

group_codes = get_group_code()

def get_themes_code():
    themes = dataset['Тема'].unique()
    dict = {themes[i]:i for i in range(0, len(themes))}
    return dict

themes_codes = get_themes_code()

y1_test = dataset_test['Группа тем']
y1_test = np.array(y1_test.replace(group_codes))
y2_test = dataset_test['Тема']
y2_test = np.array(y2_test.replace(themes_codes))

y1_train = dataset_train['Группа тем']
y1_train = np.array(y1_train.replace(group_codes))
y2_train = dataset_train['Тема']
y2_train = np.array(y2_train.replace(themes_codes))



dataset_train.shape

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\skinn\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


(49296, 4)

In [17]:
import torch
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")
model = AutoModel.from_pretrained("cointegrated/LaBSE-en-ru")

list = []

for s in x:
    s = preprocess_text(s)
    encoded_input = tokenizer(s, padding=True, truncation=True, max_length=256, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
    emb = model_output.pooler_output
    list.append((emb)[0].numpy())

x_embeddings = np.asarray(list)

#сохранить embeddings
np.save(f'embeddings_original', x_embeddings)

' 22554\nimport torch\nfrom transformers import AutoTokenizer, AutoModel\n\ntokenizer = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")\nmodel = AutoModel.from_pretrained("cointegrated/LaBSE-en-ru")\n\nlist = []\n\nfor s in x:\n    s = preprocess_text(s)\n    encoded_input = tokenizer(s, padding=True, truncation=True, max_length=256, return_tensors=\'pt\')\n    with torch.no_grad():\n        model_output = model(**encoded_input)\n    emb = model_output.pooler_output\n    list.append((emb)[0].numpy())\n\nx_embeddings = np.asarray(list)\n\n#сохранить embeddings\nnp.save(f\'embeddings_original\', x_embeddings)\n'

In [18]:
#загрузить embeddings
x_embeddings = np.load('embeddings_original.npy')
x_embeddings_test = x_embeddings[:test_count]
x_embeddings_gpt1 = np.load('embeddings_gpt1.npy')
x_embeddings_gpt2 = np.load('embeddings_gpt2.npy')
x_embeddings_gpt3 = np.load('embeddings_gpt3.npy')
x_embeddings_gpt4 = np.load('embeddings_gpt4.npy')
x_embeddings_gpt5 = np.load('embeddings_gpt5.npy')
x_embeddings_gpt6 = np.load('embeddings_gpt6.npy')

x_embeddings_train = np.concatenate([x_embeddings[test_count:], x_embeddings_gpt1, 
                                     x_embeddings_gpt2, x_embeddings_gpt3, x_embeddings_gpt4, 
                                     x_embeddings_gpt5, x_embeddings_gpt6])
x_embeddings_train.shape

(49296, 768)

In [19]:
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

x_group_train, x_group_test, y_group_train, y_group_test = x_embeddings_train, x_embeddings_test, y1_train, y1_test

x_theme_train, x_theme_test, y_theme_train, y_theme_test = x_embeddings_train, x_embeddings_test, y2_train, y2_test

x_theme_train = np.hstack((x_theme_train, y_group_train.reshape(-1, 1)))
x_theme_test = np.hstack((x_theme_test, y_group_test.reshape(-1, 1)))

In [20]:
group_model = SVC(C = 3)
group_model.fit(x_group_train, y_group_train)

y_group_pred = group_model.predict(x_group_test)

print('F1_GROUP: ', round(f1_score(y_group_test, y_group_pred, average='weighted'), 3))

F1_GROUP:  0.833


In [21]:
theme_model = SVC(C = 4)
theme_model.fit(x_theme_train, y_theme_train)

y_theme_pred = theme_model.predict(x_theme_test)

print('F1_THEME: ', round(f1_score(y_theme_test, y_theme_pred, average='weighted'), 3))

In [ ]:
from joblib import dump, load
#dump(group_model, 'group_model.joblib')
#dump(theme_model, 'theme_model.joblib')


['theme_model.joblib']

### F1_GROUP:  0.835
### F1_THEME:  0.744 0.621
